In [ ]:
from ultralytics import YOLO
import cv2

# Cargar el modelo YOLO entrenado
model = YOLO("./runs/detect/train2/weights/best.pt")

# Cargar el video de entrada
video_path = "./assets/videos/dal-lac1.mp4"
cap = cv2.VideoCapture(video_path)

# Obtener detalles del video
fourcc = cv2.VideoWriter_fourcc(*"avc1")  # Codec para el video de salida
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Configurar el video de salida
output_path = "video_resultado.mp4"
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Procesar cada frame del video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Realizar detección en el frame
    results = model(frame)

    # Dibujar las detecciones en el frame
    for result in results[0].boxes.data.tolist():  # Obtener los resultados como lista
        x1, y1, x2, y2, conf, cls = result  # Coordenadas, confianza y clase
        cls = int(cls)

        # Filtrar solo por las clases deseadas
        if model.names[cls] in ['player', 'referee', 'ball'] and conf > 0.35:

            label = f"{model.names[cls]} {conf:.2f}"
            player_position = (int((x1 + x2) / 2), y2)
            player_position_label = f"x:{int(player_position[0])} y:{int(player_position[1])}"

            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (100, 200, 100), 2)
            cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (100, 200, 100), 2)
            cv2.ellipse(frame, (int(player_position[0]), int(player_position[1])), (20, 10), 0, 0, 360, (0, 0, 255), -1)
            cv2.putText(frame, player_position_label, (int(player_position[0]) - 50, int(player_position[1]) + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    # Mostrar el frame procesado en pantalla
    cv2.imshow('Detecciones', frame)

    # Escribir el frame procesado en el video de salida
    out.write(frame)

    # Esperar 1ms para salir si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar recursos
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Video procesado guardado en {output_path}")

